# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome(executable_path='/Applications/Google Chrome.app/Contents/MacOS/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

In [6]:
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print('網頁更新中...')
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('到達頁面底端')
        break
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 影／房屋風水7大禁忌！一個小撇步化解　租屋族也可輕鬆做到　

看房賞屋，除了建材好壞、格局搭配等要仔細看外，其實風水上的禁忌也要特別注意，雖然現代人可能越來越沒有拘束這些風水習俗，但房產網紅「建築先生」還是建議，寧可信其有不可信其無喔，畢竟房子內要住的是自己，花錢也要住的安心！就算是租屋的朋友，也要特別注意，盡量避免租到這類型的房屋喔！
房產網紅Jackson就在《建築先生 Jackson敲敲門》節目中介紹了7種禁忌風水：
Jackson也提醒，雖然現代人可能越來越沒有拘束這些風水習俗，但還是建議寧可信其有不可信其無，畢竟房子內要住的是自己，花點小錢可以住得更安心，何樂而不為呢？
►勝過高鐵撿到3百萬的快感
----------------------------------------------------------------------

[1] 意外邊際效應！美伊緊張：投資者轉往債券避險「30年房貸利率穩低」

▲美國低端房產市場，可能得利於美國空襲伊朗的邊際效應。（圖／pixabay）
記者張靖榕／綜合外電報導
由於隔夜債券市場發生擠兌，美國30年期固定房貸利率以降低至近一個月內的低點。而擠兌的情況是受到了遠在中東地區突發的美伊緊張局面，導致投資客紛紛轉往風險較低的債券市場。
不動產抵押貸款證券（MBS）雖然影響房貸利率的升降，但他們並未緊跟著美國10年期國債的變化而迅速改變。房產網站《每日房貸新聞》首席營運長葛拉罕（Matthew Graham）表示，「多虧美國和伊朗的區域性緊張，國債的收益率又降低了」，由於MBS的適度微調能夠改善房貸利率，因此今年1月的最低利率落在3日到5日。
儘管國債收益率已有所回升，房貸利率今年初仍持續下降，而由於MBS屬於較短線操作，近日的表現會較國債收益率要佳。葛拉罕指出，「30年期固定利率若來到3.75％，將會是最完美的情況，而2日就已經來到3.625％了。
30年期固定利率平均已經比2019年要低上87個基礎點，等於幫助買家省下了很大一筆錢，尤其是在低端房產市場。房地美（Freddie Mac）首席經濟學家凱特（Sam Khater）表示，現階段的低房貸利率環境，加上勞

▲屏東中洲建設公司「拾光居」 。（圖／中洲建設公司提供）
記者洪靖宜／高雄報導
過去屏東的房子設計總讓人覺得建築外觀的創意較為不足，但近年開始出現變化，除外觀更多元、活潑外，就連景觀設計也愈來愈受到重視，更得到國際大獎肯定，屏東中洲建設公司在麟洛鄉推出的透天案「拾光居」獲柏林設計獎、紐約設計獎，由於受好評，在萬丹推出「拾光居2」一樣以以大量的玻璃磚來做於立面，不只採光好，還保有隱私。
▲「拾光居」以大量玻璃磚來做坐立面 。（圖／中洲建設公司提供）
屏東中洲建設公司在麟洛鄉推出的透天案「拾光居」外觀全用了玻璃磚，這個在台灣是非常少見，配合晚上的燈光計畫，其實是非常特別。讓他們獲得2018年柏林設計獎、紐約設計獎雙銀獎。這兩個國際級的獎項，那全台灣除了中洲建設之外，還有兩個在台北的建商得獎。
▲不僅採光好又保有隱私性。（圖／中洲建設公司提供）
查了實價登錄，同為得獎建商，比起另一間在台北的大樓案，只要用兩坪多一點價格就可以買得起就能買到中洲建設的「拾光居」一整棟，業者自豪表示，能在有限預算內打造出同等具有國際級水準的建築，就覺得我們當初知道全台灣只有三家建商得獎，滿開心的。
由於好評不斷，現在在萬丹推出「拾光居2」，一樣以以大量的玻璃磚來做於立面，不只採光，還保有隱私，在玻璃磚牆做了噴砂玻璃，讓同時保有採光光灑了進來，也測試過任何角度，隱私不受影響。
----------------------------------------------------------------------

[12] 我國負債比率連六年下降　大減5.3個百分點

▲財政部揭牌儀式。（圖／記者陳韋帆攝）
記者吳靜君／台北報導
財政部國庫署表示，2018年度我國依國際貨幣基金發布之政府財政統計手冊計算一般政府負債新台幣6兆7,105億元，占2018年度國內生產毛額(GDP)比率36.58%。根據國庫署發布，我國2018年負債比率不僅是G20中排名第5低、負債比連續六年下降，六年間也大減了5.3個百分點，顯示我國控制債務有成。
國庫署表示，與國際貨幣基金公布G20不含歐盟本體等19個國家比較，我國負債比低於其中15個國家，包含日本、法國、韓國、中國大陸等主要國家，僅高於4個國家，俄羅斯、阿拉伯、印尼與土耳其。
我國負債比率已經連續六年下降，從2013至2018年已經從41.9%下降到36.5

▲LINE新春推出「金鼠集好運」集章活動。（圖／LINE提供）
記者姚惠茹／台北報導
LINE今年特別在新春推出「金鼠集好運」集章活動，只要從即日起活動指定任務即可獲得集章，達到每階段指定累積集章數量即可兌換好禮，獎品價值總計超過LINE POINTS 500萬點、LINE代幣600萬點，任務包括LINE貼圖、LINE GAME、LINE購物、LINE旅遊。
LINE這次活動共有四個服務，包括LINE貼圖「錢錢鼠不完」、LINE GAME「來學院辦桌」、LINE購物「福袋戳戳樂」、LINE旅遊「走春求好籤」，活動期間內完成各系列活動的指定任務，將透過「LINE 金鼠集好運」官方帳號發送獲得集章通知或好禮中獎通知，指定任務重置時間為每日零點。
▲LINE新春推「金鼠集好運」集章活動，每日可到LINE旅遊求鼠年旅遊運勢籤。（圖／LINE提供）
其中LINE貼圖為每日報到集好禮、LINE GAME則是完成指定任務，而LINE旅遊需要用戶至點進走春求好籤頁面，每日來求鼠年旅遊運勢籤，只要抽中上上籤即可在金鼠集好運集點卡中，集到一格好運章，還有春節賺點不打烊，機票/住宿/體驗/跟團點數回饋無上限，以及agoda住宿/KKday體驗/Tripresso跟團折扣碼不限量任你領，甚至是日本電器/藥妝店優惠券最高19%優惠的活動。
至於LINE購物的「福袋戳戳樂」，只要每日至「LINE購物」完成指定任務，戳中「驚喜福袋」即可獲得一格好運章，不過單一LINE帳號每日僅可透過任務獲得一格好運章，因此每日最多可以在走春求好籤頁面獲得一格好運章，活動七日最多獲得七格，另外還有「新春搶福袋，限量1折起」和「賺點不打烊，最高10%回饋」，以及「領券再購物，最低84折起」等好康活動。
LINE表示，用戶只要進入各服務參與活動，每完成一個活動即可集一格好運章， 集滿4個章就有機會兌換LINE代幣5枚，累積集滿12章和20章各有機會兌換LINE POINTS 3點，如果七天活動期間天天登入，完成累積28章就有機會兌換獲得LINE POINTS 60點。
▲LINE新春推「金鼠集好運」集章活動，每日可到LINE購物玩福袋戳戳樂。（圖／LINE提供）
----------------------------------------------------------------------

[23] 大

記者姚惠茹／台北報導
鴻海（2317）愛心嘉年華今（22）日在員工眷屬抽獎時，發生意外插曲，創辦人郭台銘偕同夫人曾馨瑩上台加碼，先送出10個百萬紅包，接著加碼3個250萬大獎，但要求要在場員工才能領，結果抽到一位趕搭高鐵的員工，由於不在現場，錯失250萬大獎的中獎資格。其他得獎者中，有位自武漢返台員工被抽中，雖然沒有到場，但因配合公司自主管理無法出席，因此幸運得到250萬元。
▲鴻海愛心嘉年華。（圖／記者姚惠茹攝）
鴻海今年尾牙由董事長劉揚偉主持，並在員工眷屬獎抽出200個10萬紅包後，大方加碼200個10萬紅包，而創辦人郭台銘更是一路往上加碼，共送出10個100萬紅包、3個250萬紅包，以及最大獎500萬紅包2個，總獎金達6750萬元。
首先夫人曾馨瑩順利抽出10個百萬紅包，接著在張菲鼓動之下，郭台銘再喊出3個250萬，沒想到抽完第2位後沒有人上台，結果傳來該位同仁正在搭捷運趕高鐵前往高雄，意外錯失250萬大獎，更讓郭台銘當場笑說「沒辦法，現在高雄問題比較多」。
但是抽到第3位250萬得主的時候，同樣沒有人上台，馬上有人喊說這是武漢返台員工，因為集團防疫作業，而被要求在家參與尾牙直播，郭台銘則霸氣提到要為這名同仁保留中獎資格，並強調為了不要增加感染風險，武漢的同仁要保留。
最後郭台銘更是喊出最大獎500萬元，而且要抽出2位幸運得主，結果抽中前面4個都不在場，一直到重抽第5位才順利抱回500萬元大獎，其中一位員工更在上台時掩不住喜悅地表示自己要把這筆錢存起來。
----------------------------------------------------------------------

[29] iPhone賣太好！蘋果要台積電Q1提高產能　加快A13處理器製造

記者周康玉／台北報導
由於iPhone 11和iPhone 11 Pro在去年秋季首次亮相就獲得銷售佳績，特別是在中國大陸市場的強勁需求，比前（2018）年的萎縮市況銷售更好。即使還沒正式步入5G，iPhone需求也一直超出市場和蘋果的期望，蘋果因此要求組裝合作夥伴提高最新一代產品的產量。彭博引述一名消息人士指出，iPhone 11因為搭載了LCD螢幕，成為帶動需求的特殊動力。
由於市場傳出，今年春季可能會推出主打中階的新款iPhone SE，並且搭載與iPhone 11相同的處理器，將於

▲主計總處國勢普查處副處長潘寧馨。（圖／記者余弦妙攝）
記者余弦妙／台北報導
根據調查顯示，12月勞動力人數為1,197萬人，較上月減少3千人或0.02%，較107年同月則增5萬4千人或0.45%。108年勞動力人數平均為1,194萬6千人，較上年增加 7 萬 2 千人或 0.60%。12月勞動力參與率為 59.22%，較上月下降0.03個百分點，較上年同月則升 0.13 個百分點。
若以年齡層來看，去年15到19歲失業率平均為9.22%，20至24歲者12.27%，較2018年分別上升0.76個及0.29個百分點，原因是多為初次尋職，且處職場調適階段，導致失業率較高。
對於今年的整體失業率的展望，主計總處國勢普查處副處長潘寧馨表示，通常不太會針對失業率進行預測，主要還是要看整體經濟情勢，與企業用人情況而定，但能確定的是季節性因素是不變的。
至於世界其他主要國家失業率，法國8.4%、加拿大5.6%、英國3.7%、南韓3.6%、美國3.5%、香港3.2%、新加坡2.3%、日本2.2%。
----------------------------------------------------------------------

[39] 金仁寶下令所有廠區量耳溫、戴口罩　武漢廠區對外僅能視訊

▲康舒淡水新大樓動土。（圖／記者周康玉攝）
記者周康玉／台北報導
小年夜前夕，電源供應一線大廠康舒今（22）日上午進行淡水廠區新廠房新建工程動土典禮。由於同集團在武漢有一個廠區，市場關心，武漢肺炎疫情對工廠影響。
金仁寶集團董事長許勝雄表示，所有廠區內部已啟動防護措施，上班戴口罩，員工上班也要量耳溫，並且在兩天內會把影像測溫設備架設好，廠區內的同仁都不能與武漢廠區人員往來，一切業務或開會都用視訊。
在產能因應部分，許勝雄表示，先前集團為了因應農曆長假把2月的產量提前到1月生產，因此零組件供貨無虞，即使疫情發生，生產不受影響，一個月內都沒問題。然而，如果疫情超過一個月，才可能供貨困難。
金仁寶在全球有55個工廠，其中中國大陸就有22個廠，其中有一個廠在武漢。目前集團已經響應中國大陸政府「不希望外地人到武漢，也不希望武漢人到外地」政策，因此與武漢廠區人員開會僅能用視訊。
--------------------------------------------------------

▲ 新台幣匯率在豬年全年升值2.4%。（圖／路透）
記者李瑞瑾／台北報導
今（22）日為農曆豬年銀行最後營業日，也是匯市封關以及最後換新鈔的日子。台北外匯市場新台幣匯率收在30.006元兌1美元、貶值0.8分，成交金額8.43億美元。
另外，新台幣的升值也讓民眾換匯變得更便宜了！根據台銀歷史牌告匯價，日圓在狗年封關時換匯價為0.2857元，今年則為0.276元，便宜了3.39%；美元匯價便宜了2.45%、歐元便宜了5.49%、英鎊便宜了2.86%、人民幣也便宜了4.65%。
但另一方面，表現相對強勢的泰銖，換匯價則是貴了0.59%。
新台幣兌美元今天開盤為30.01元兌1美元、貶值1.2分，隨後持續在30元關卡附近拉鋸，早盤多處於貶勢、午盤過後則轉為走升，直到尾盤轉為貶值，以30.006元兌1美元作收，盤中最高29.975元、最低30.024元。
今日匯市封關後，台北匯市也與台股一樣，將於1月30日開紅盤。
----------------------------------------------------------------------

[47] 捷運新莊站出站到家「永鼎帝京」百年富貴核心　千坪絕版地王

圖、文／永鼎帝京提供
坐擁千坪不稀奇！能坐擁一塊傳富百年的千坪地王，才是打著燈籠也找不到的稀有福氣。【永鼎帝京】以座落大漢溪畔、明清時代「一府二鹿三新莊」發跡核心、北台灣開發最早的政經、文教、聚富主場「中正大道」，加上台北碩果僅存文史寶地「廟街」、百年菁英名校、公部門，金融行庫、捷運簇擁，淬煉地王能量，聚足百年新莊繁華！永鼎建設以好地難得一遇，禮聘豪宅團隊，打造戶戶邊間、大窗大戶大景規格，為坐鎮新莊正都心，獻上凌駕地標級的鉅擘！
新莊正都心最後壓軸  1833坪傳世寶庫
基地前身為封藏數十載「警政宿舍」官地釋出。屢獲金石獎殊榮的永鼎建設，深知百年好地難得一遇，特為這塊坐擁中正大道首排、四面臨路大地，聯手「仁愛帝寶」豪宅建築大師－王裕華、「台北101」摩天建築結構翹楚－永峻工程、「元利信義聯勤」園藝景觀高手－蘇瑞泉、以及備受業界推崇的久年營造等頂尖行家團隊，以「傳世寶庫」為軸，打造22層地標建築，中庭園林勝景、金幣家徽迎賓，結合基座巴西黃金石精工，如一座超高規的雍容三合院，完美滿足世代傳承。
大面寬客廳全景大窗 戶戶邊間大河大景
大面開窗、三面採光，為坐

▲亞太電信董事長呂芳銘。（圖／記者姚惠茹攝）
記者邱倢芯／台北報導
鴻海集團今（22）日在南港展覽館舉行尾牙，會中亞太電信董事長呂芳銘接受媒體訪問時提到，亞太電之所以會放棄標3.5GHz頻段，真的就是標金太高、太離譜。
呂芳銘指出，第一階段競標時，對於3.5GH頻段考慮很多，不過最後回到集團策略考量，因此放棄標此一頻段。
「當天決標後，亞太馬上接到友商主動聯繫洽詢合作機會。」呂芳銘透露，決標當天，就有其他電信業者詢問雙方合作的機會。而且電信三共（共頻、共網、共建）計畫法條並沒有特別規定業者需要擁有頻譜才能合作三共，因此未來亞太會以合作的方式來提供消費者服務。
----------------------------------------------------------------------

[57] 長榮航飛安再下一城　德國評比躋身前十大全球最安全航空公司

▲德國航空安全資料中心今（22）日公佈2020年全球最安全航空公司排名，長榮航空的飛安名列全球第九名。（圖／長榮航空提供）
記者林淑慧／台北報導
德國航空安全資料中心今（22）日公佈2020年全球最安全航空公司排名，長榮航空的飛安名列全球第九名，這是繼本月初獲得航空專業評鑑網站AirlineRatings.com評選為全球前十大最安全航空公司之後，再度榮膺國際飛安機構的肯定。
JACDEC為專業航空安全資料分析機構，每年針對全球各大航空公司之飛行年數、機隊年齡、航班起降次數、近30年來航機意外事故統計，以及國際航空運輸協會(IATA)的飛安稽核結果等33項標準進行評比，遴選出全球100家最安全航空公司。
長榮航空表示，公司致力於落實完善的飛航安全訓練，除了引進各機型全飛行模擬機、客艙緊急逃生訓練艙、滅火訓練艙等設施，提升員工在緊急狀況下的應變能力，藉以維持高標準的飛安品質。
另外，自2003年國際航空運輸協會(IATA)創立IOSA航空作業安全認證制度以來，長榮航空已經連續8次以「零缺點」的佳績，通過每二年一次的IOSA查核認證。
展望後市，法人預估在787新世代飛機投入高運量及中長程航線下，長榮航空可望增加產品競爭力，另導入新一代票價系統，以差異化票價產品滿足不同客群，以提升品質來增加航班收益，全年獲利可期。
依據財報，長榮航空2019年第3季稅後純益為12.71億元，雖然仍較前年同期腰斬，但比起去

▲左起為夏普中國總經理林瑞祥、鴻海副董事長李傑、創辦人郭台銘、董事長劉揚偉、總財務長黃秋蓮。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
鴻海（2317）今（22）日在南港展覽館舉辦愛心嘉年華，創辦人郭台銘致詞憂心武漢肺炎蔓延，呼籲台灣人暫時不要前往武漢，並提到鴻海在武漢有工廠，因此內部已經在開會研擬「視訊會議」取代武漢同仁「年後返工」。
郭台銘表示，武漢肺炎有幾個特色， 第一是傳染性非常快，第二是現在正處於爬坡期，意味著接下來將會等比倍數增加，而且台灣目前已經有一例確診，再加上鴻海在武漢有工廠，所以內部已經在第一時間開會商量，希望武漢回來的同仁和眷屬都不要來參加尾牙。
郭台銘向員工呼籲，所有到過武漢的員工，請儘速向公司登記回家休息，因為武漢肺炎傳染快速，不會亞於當年的SARS，建議大家過年最好不要到大陸去遊玩，而且口罩、藥品、檢驗器材不足，生產更趕不上傳染的速度，並希望大家今年要過一個好年，所以一定要注意自己的健康防疫。
郭台銘強調，鴻海內部已經開會，針對疫情的發展，討論武漢同仁年後復工，由事業主管決定，看是要透過視訊遙控讓工作繼續進行，還是透過視訊會議系統管控，有助於讓疫情受到控制，並鼓勵員工藉此留點時間在台灣，多生一點鴻海寶寶。
----------------------------------------------------------------------

[69] 謝金河：台灣2019年的經濟成長率...全部人都猜錯了！

▲財信董事長謝金河。（圖／ETtoday資料照）
記者謝仁傑／綜合報導
謝金河昨（21）日在臉書指出，行政院主計總處公布去年第四季GDP成長率是3.38％，比預估的3.04％還高出許多；一方面是民間消費增加2.94％，另一個是資本形成的政府投資及民間投資，再加上存貨變動調整，增加10.72％。台灣在第四季經濟增長拉高到3.38％，也推升2019年全年GDP成長率從預期2.64％再拉高到2.73％，去年的經濟表現穩居四小龍之首。
謝金河指出去年台灣全年GDP成長率2.73％，高於南韓的2％，也比新加坡的0.7％及香港的負1.3％出色很多，與去年上半年標準普爾等機構，皆預測台灣在2019年經濟成長率會居四小龍之末，完全相反。
國內所有經濟研究機構，去年估最高的是中研院的2.45％，最差的是星展銀行估1.9％，中華經濟研究院2.14

▲台經院。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台灣經濟研究院今（22）日公布「國內總體經濟預測暨景氣動向調查」，針對2020年總體經濟而言，台經院預測2020年國內經濟成長率為2.67%，較2019年11月預測上修0.22個百分點。
台經院表示，觀察近期國際經濟情勢，儘管中美雙方簽署第一階段貿易協議，有助於緩解市場觀望氣氛，然主要國家各項經濟先行指標仍顯疲弱，顯示全球經濟需求尚未明顯回溫。
內需方面，進入產業淡季，且部分科技供應鏈進入庫存調整，但受惠於5G通訊等新興應用熱潮，廠商持續擴增在台產能，令電子機械業廠商對當月景氣仍維持一定的樂觀程度。傳統產業則受惠於美中貿易情勢趨於緩和，國際油價、鋼價回升，有助緩解市場觀望氣氛，使得傳統產業對當月景氣與未來半年景氣較先前調查樂觀。
服務業方面受惠於選舉造勢、企業尾牙、耶誕旺季及跨年活動，加上12月台股表現亮眼，拉抬零售業、餐旅業與證券業對當月景氣看法轉強。
營建業方面受惠於年底列管公共工程標案陸續結案驗收，使得工程認列集中入帳，加上各地交屋潮挹注、自住客買盤持續浮現，拉抬六都建物買賣移轉件數的數據，此均使得12月營建業表現優於原先預期，至於未來半年營建業廠商對景氣看法皆以持平看法居多。
根據本院調查結果，經過模型試算後，12月製造業、服務業與營建業營業氣候測驗點同步走高，其中製造業營業氣候測驗點連續兩個月上揚，服務業結束先前連續四個月下滑態勢轉為上揚。
展望今年，台經院指出，近期國際預測機構如國際貨幣基金組織(IMF)、經濟學人(EIU)、世界銀行(WB)與IHS Markit所公布的更新數據觀察，儘管多數預測機構下修2020年全球經濟成長表現，但下修幅度有限，加上近期中美雙方貿易緊張情勢降溫，有助於貿易與製造業活動可望觸底回升。
此外，隨著多國推出財政政策搭配貨幣寬鬆以刺激經濟，新興市場及開發中經濟體表現將可望較2019年改善，抵銷了美中經濟持續下滑的風險。因此，台經院預估，2020年國內經濟成長率為2.67%，較2019年11月預測上修0.22個百分點。
----------------------------------------------------------------------

[79] 康舒淡水新大樓動土　預計2021年產能倍增、創造300個研發工作機會

▲康舒新大樓動土典禮。（圖／記者周

圖文／CTWANT
中信房屋依據新北市政府民政及實價資料進行統計，捷運板橋站是沿線各站中平均單價最高的站點，2019年華廈住宅大樓實價均價每坪逾56萬元，公寓亦有42萬元之譜。中信房屋研展室副理張漢超表示，板橋站位於新板特區商圈內、比鄰新北市政府，由商圈帶動房價的型態明顯，因此無論較新的華廈住宅大樓或傳統的無電梯公寓，房價都同時聯袂受惠。
相關研究曾指出民眾通常願意步行搭乘捷運的時間往往不會超過10分鐘，換算距離約在500～700公尺，因此若篩選出捷運站周圍500公尺的人口數及戶數，則可看出車站對居民服務的效益。統計顯示捷運景安站不僅是人口數最多，成交件數也是沿線各站之最，至於捷運板新站則是戶數最多的站點。
►「2020愛妳愛妳」～超殺新年禮盒來了！
----------------------------------------------------------------------

[89] 全台100強企業名單曝光！和泰員工好幸福　做1年領2年薪

圖文／鏡週刊
和泰車是台灣上市公司中，除了聯發科等科技業之外，對員工福利最好的公司之一，光是每年領的薪水與獎金，不少傳產企業都遠遠比不上。
 
和泰車去年銷售成績極佳，前三季獲利88.51億元，每股獲利更衝上16.2元，法人圈認為全年有機會賺超過2個股本。
公司賺錢，也讓和泰車員工荷包滿滿，根據證交所2018年的薪資福利調查結果，和泰車非主管職的平均薪資以199萬元穩居傳產業第一，遙遙領先同業，除了固定3個月的年終獎金之外，這5年來，幾乎年年都發放9個月以上的績效獎金，年終加績效獎金，就等於多一年的薪水。
「在和泰工作一年，就可以領兩年薪水。」在和泰車已待超過30年的和泰車發言人協理劉松山說，「新進人員若績效好，3年左右就能年薪百萬。」
不只薪水做1年賺2年，和泰對員工福利也很好，每年光三節獎金、旅遊補助再加上子女教育補助就超過10萬元。「總經理（蘇純興）常說，人是和泰車最大的資產。」和泰主管笑著說。
►「2020愛妳愛妳」～超殺新年禮盒來了！
►「2020愛妳愛妳」～超殺新年禮盒來了！
----------------------------------------------------------------------

[90] 上市櫃公司持股5%股東全都露　1年揭露5次最快第1季上路

▲金管會研